In [2]:
import joblib

# Load the model
rf = joblib.load('random_forest_model.pkl')

Rebuild Feature Columns

In [ ]:
import pandas as pd
from fastai.tabular.all import *

test_df = pd.read_csv('test.csv')

test_df['Age'] = test_df['Age'].dropna()
test_df['Child'] = test_df['Age'].between(0, 16, inclusive='both')
test_df['Adult'] = test_df['Age'].between(17, 45, inclusive='both')
test_df['Senior'] = test_df['Age'].between(46, 100, inclusive='both')
test_df['Fare_log'] = np.log1p(test_df['Fare'])
test_df['cabin_height'] = test_df['Cabin'].str[0].fillna('U')
test_df['Embarked'] = test_df['Embarked'].fillna(0)
procs = [Categorify, FillMissing, Normalize]
cat_names  = ['Sex','Embarked','cabin_height','Pclass']
cont_names = ['Fare_log','Child','Adult','Senior','SibSp','Parch']

tp = TabularPandas(
    test_df, 
    procs=procs,
    cat_names=cat_names, 
    cont_names=cont_names,
    y_names='Survived',
    y_block=RegressionBlock(),
    splits = RandomSplitter(0.2)(range_of(test_df)),
    )

df = tp.items
df[cont_names] = df[cont_names]
X_test = df[cont_names + cat_names]


c:\Code\Local Code\titanic_survive\.venv\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


TypeError: 'method' object is not subscriptable

In [14]:
X_test.columns.tolist()
X_test = X_test.drop('Fare_log_na', axis=1)
X_test.columns.tolist()

['Fare_log',
 'Child',
 'Adult',
 'Senior',
 'SibSp',
 'Parch',
 'Sex',
 'Embarked',
 'cabin_height',
 'Pclass']

In [15]:
from sklearn.ensemble import RandomForestClassifier
preds = rf.predict(X_test)

In [16]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': preds
})

submission.to_csv('rfm_submission.csv', index=False)